In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
!pip install yfinance
import yfinance as yf

In [ ]:
block = yf.Ticker('XYZ')
block

In [ ]:
block_data = block.history(start = '2020-01-01', end = '2024-12-31', interval = '1d', auto_adjust = False)
block_data

In [ ]:
block_data.reset_index(inplace=True)
block_data

In [ ]:
block_data.isnull().sum()

In [ ]:
block_data['Stock Splits'].sum()

In [ ]:
block_data['Dividends'].sum()

In [ ]:
block_data.drop(['Dividends','Stock Splits'], axis = 1, inplace = True)
block_data

In [ ]:
block_data['Daily Return'] = block_data['Adj Close'].pct_change(1) * 100
block_data

In [ ]:
block_data['Daily Return'].fillna(0)
block_data

In [ ]:
block_data.describe().round(2)

In [ ]:
import matplotlib.pyplot as plt

!pip install seaborn
import seaborn as sns

!pip install plotly
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=block_data['Date'],
                open=block_data['Open'],
                high=block_data['High'],
                low=block_data['Low'],
                close=block_data['Adj Close'])])

fig.show()

In [ ]:
fig = px.line(block_data, x='Date', y='Volume', title='xyz volume')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
fig = px.line(block_data, x='Date', y='Daily Return', title='xyz daily return')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
block_data.describe()

In [ ]:
std_dev = block_data['Daily Return'].std()
std_dev

In [ ]:
bins = [-np.inf, -3*std_dev, -2*std_dev, -1*std_dev, 1*std_dev, 2*std_dev, 3*std_dev, np.inf]
labels = ['<-3σ', '-3σ to -2σ', '-2σ to -1σ', '-1σ to 1σ', '1σ to 2σ', '2σ to 3σ', '>3σ']
block_data['Category'] = pd.cut(block_data['Daily Return'], bins=bins, labels=labels)

category_counts = block_data['Category'].value_counts().sort_index()
plt.figure(figsize=(8, 8))
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', pctdistance = 1.05, labeldistance = 1.2, startangle = 140, colors=plt.cm.Paired(range(len(labels))))
plt.title('Distribution of Daily Percentage Moves for XYZ Stock')
plt.show()

In [ ]:
pip install pandas pandas_datareader

In [ ]:
from pandas_datareader import data as pdr
import datetime

In [ ]:
block_competitors = ['PYPL', 'AFRM', 'GPN', 'FOUR', 'XYZ']
start_date = '2022-01-01'
end_date = '2024-12-31'

In [ ]:
close_price_data = yf.download(block_competitors, start=start_date, end=end_date)['Close']
print(data.head())

In [ ]:
df_long = close_price_data.reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Close Price')
df_long

In [ ]:
fig = px.line(df_long, x='Date', y='Close Price', color='Ticker', title='Stock Closing Prices')
fig.show()

In [ ]:
daily_returns = (close_price_data.iloc[1:] - close_price_data.iloc[:-1].values) / close_price_data.iloc[:-1].values
daily_returns_df = pd.DataFrame(daily_returns, columns=close_price_data.columns, index=close_price_data.index[1:])
daily_returns_df

In [ ]:
daily_returns_df_px = daily_returns_df.reset_index().melt(id_vars="Date", var_name="Ticker", value_name="Return")
fig = px.line(daily_returns_df_px, x="Date", y="Return", color="Ticker", title="Stock Return Over Time")
fig.show()

In [ ]:
fig = px.histogram(daily_returns_df)
fig.update_layout(
    title="Distribution of Daily Stock Returns",
    plot_bgcolor="white"
)
fig.show()

In [ ]:
plt.figure(figsize = (10, 8))
sns.heatmap(daily_returns_df.corr(), annot = True, cmap = 'crest')

In [ ]:
sns.pairplot(daily_returns_df);

In [ ]:
def price_scaling(raw_prices_df):
    scaled_prices_df = raw_prices_df.copy()
    for col in raw_prices_df.columns:
        scaled_prices_df[col] = raw_prices_df[col] / raw_prices_df.iloc[0][col]
    return scaled_prices_df

In [ ]:
price_scaling(close_price_data)

In [ ]:
fig = px.line(price_scaling(close_price_data))
fig.show()

In [ ]:
import random

def generate_random_weights(n):
    weights = []
    for i in range(n):
        weights.append(random.random())
    weights = weights/np.sum(weights)
    return weights

In [ ]:
weights = generate_random_weights(5)
print(weights)

In [ ]:
weights.sum()

In [ ]:
close_price_data

In [ ]:
portfolio_df = close_price_data.copy()
scaled_df = price_scaling(portfolio_df)
scaled_df

In [ ]:
initial_investment = 1000000
for i, stock in enumerate(scaled_df.columns[0:]):
    portfolio_df[stock] = weights[i] * scaled_df[stock] * initial_investment

portfolio_df.round(1)

In [ ]:
def asset_allocation(df, weights, initial_investment):
    portfolio_df = df.copy()
    scaled_df = price_scaling(df)
    for i, stock in enumerate(scaled_df.columns[0:]):
        portfolio_df[stock] = scaled_df[stock] * weights[i] * initial_investment

    portfolio_df['Portfolio Value'] = portfolio_df[portfolio_df != 'Date'].sum(axis=1, numeric_only = True)
    portfolio_df['Daily % Return'] = portfolio_df['Portfolio Value'].pct_change(1) * 100
    portfolio_df.replace(np.nan, 0, inplace = True)

    return portfolio_df

In [ ]:
n = len(close_price_data.columns)

print('Number of stocks under consideration: {}'.format(n))
weights = generate_random_weights(n).round(5)
print('Portfolio Weights: {}'.format(weights))

portfolio_df = asset_allocation(close_price_data, weights, 1000000)
portfolio_df.round(2)

In [ ]:
fig = px.line(portfolio_df, x=portfolio_df.index, y='Daily % Return', 
              title="Daily Percentage Return",
              labels={'Daily % Return': 'Daily Return (%)', 'index': 'Date'})
fig.show()


fig = px.line(portfolio_df, x=portfolio_df.index, y= ['AFRM', 'FOUR', 'GPN', 'PYPL', 'XYZ'], 
              title="Portfolio Value",
              labels={'Daily % Return': 'Daily Return (%)', 'index': 'Date'})
fig.show()


fig = px.line(portfolio_df, x=portfolio_df.index, y='Portfolio Value', 
              title="Portfolio Value",
              labels={'Daily % Return': 'Daily Return (%)', 'index': 'Date'})
fig.show()

In [ ]:
def simulation_engine(weights, initial_investment):
    portfolio_df = asset_allocation(close_price_data, weights, initial_investment)
    return_on_investment = ((portfolio_df['Portfolio Value'].iloc[-1] -
                             portfolio_df['Portfolio Value'].iloc[0])/
                             portfolio_df['Portfolio Value'].iloc[0]) *100

    portfolio_daily_return_df = portfolio_df[['AFRM', 'FOUR', 'GPN', 'PYPL', 'XYZ']]
    portfolio_daily_return_df = portfolio_daily_return_df.pct_change(1)

    expected_portfolio_return = np.sum(weights * portfolio_daily_return_df.mean()) *252

    covariance = portfolio_daily_return_df.cov() * 252
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

    final_portfolio_value = portfolio_df['Portfolio Value'].iloc[-1]

    rf = 0.0432

    sharpe_ratio = (expected_portfolio_return-rf)/expected_volatility
    return expected_portfolio_return, expected_volatility, sharpe_ratio, portfolio_df['Portfolio Value'].iloc[-1], return_on_investment

In [ ]:
initial_investment = 1000000
portfolio_metrics = simulation_engine(weights, initial_investment)

In [ ]:
print("Expected Portfolio Return: {:.2f}%".format(portfolio_metrics[0]*100))
print("Expected Portfolio Volatility: {:.2f}%".format(portfolio_metrics[1]*100))
print("Sharpe Ratio: {:.2f}".format(portfolio_metrics[2]))
print("Final Portfolio Value: ${:.2f}".format(portfolio_metrics[3]))
print("Return on Investment: {:.2f}%".format(portfolio_metrics[4]))

In [ ]:
sim_runs = 5000
initial_investment = 1000000

weight_runs = np.zeros((sim_runs, n))
sharpe_ratio_runs = np.zeros(sim_runs)
expected_portfolio_returns_runs = np.zeros(sim_runs)
volatility_runs = np.zeros(sim_runs)
return_on_investment_runs = np.zeros(sim_runs)
final_value_runs = np.zeros(sim_runs)

for i in range(sim_runs):
    weights = generate_random_weights(n)
    weight_runs[i, :] = weights

    expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i], final_value_runs[i], return_on_investment_runs[i] = simulation_engine(weights, initial_investment)
    print(f'Simulation Run = {i}')
    print(f'Weights = {weight_runs[i].round(3).tolist()}')
    print(f'Final Portfolio Value = ${final_value_runs[i]:,.2f}')
    print(f'Sharpe Ratio = {sharpe_ratio_runs[i]:.2f}')
    print('\n')

In [ ]:
sharpe_ratio_runs

In [ ]:
sharpe_ratio_runs.argmax()

In [ ]:
sharpe_ratio_runs.max()

In [ ]:
weight_runs[sharpe_ratio_runs.argmax(), :]

In [ ]:
optimal_portfolio_return, optimal_volatility, optimal_sharpe_ratio, highest_final_value, optimal_roi = simulation_engine(weight_runs[sharpe_ratio_runs.argmax(), :], initial_investment)

In [ ]:
print('Best Portfolio Metrics Based on {} Simulation Runs'.format(sim_runs))
print('    - Portfolio Expected Annual Return: {:.02f}%'.format(optimal_portfolio_return*100))
print('    - Portfolio Optimal Volatility: {:.02f}%'.format(optimal_volatility*100))
print('    - Optimal Sharpe Ratio: {:.02f}'.format(optimal_sharpe_ratio))
print('    - Optimal Portfolio Value: ${:.02f}'.format(highest_final_value))
print('    - Optimal Portfolio ROI: {:.02F}%'.format(optimal_roi))

In [ ]:
simulation_df = pd.DataFrame({'Volatility': volatility_runs.tolist(), 'Portfolio Return': expected_portfolio_returns_runs.tolist(), 'Sharpe Ratio': sharpe_ratio_runs.tolist()  })
simulation_df

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=simulation_df['Volatility'],
    y=simulation_df['Portfolio Return'],
    mode='markers',
    marker=dict(
        size=10,
        color=simulation_df['Sharpe Ratio'],  # Color by Sharpe Ratio
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Sharpe Ratio")
    ),
    text=[f"Sharpe: {sr:.2f}" for sr in simulation_df['Sharpe Ratio']],  # Hover text
    name="Portfolio Simulations"
))

# Layout Settings
fig.update_layout(
    title="Portfolio Simulation: Volatility vs Expected Return",
    xaxis_title="Volatility (Risk)",
    yaxis_title="Expected Portfolio Return",
    template="plotly_white",
    hovermode="closest"
)

# Show the plot
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=simulation_df['Volatility'],
    y=simulation_df['Portfolio Return'],
    mode='markers',
    marker=dict(
        size=10,
        color=simulation_df['Sharpe Ratio'],  # Color markers by Sharpe Ratio
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Sharpe Ratio")
    ),
    text=[f"Sharpe: {sr:.2f}" for sr in simulation_df['Sharpe Ratio']],
    name="Portfolio Simulations"
))

# Find the row with the maximum Sharpe ratio
max_idx = simulation_df['Sharpe Ratio'].idxmax()
max_point = simulation_df.loc[max_idx]

# Add an extra trace for the maximum Sharpe ratio point (the "exploded" marker)
fig.add_trace(go.Scatter(
    x=[max_point['Volatility']],
    y=[max_point['Portfolio Return']],
    mode='markers+text',
    marker=dict(
        size=20,
        color='red',
        symbol='star'
    ),
    text=[f"Max Sharpe: {max_point['Sharpe Ratio']:.2f}"],
    textposition='top center',
    name='Max Sharpe Ratio'
))

# Update layout settings
fig.update_layout(
    title="Portfolio Simulation: Volatility vs Expected Return (Max Sharpe Highlighted)",
    xaxis_title="Volatility (Risk)",
    yaxis_title="Expected Portfolio Return",
    template="plotly_white",
    hovermode="closest"
)

# Display the plot
fig.show()